# COGS 108 - Data Checkpoint

# Names

- Aasem Fituri
- Casey Hild
- Carlos van der Ley
- Jeremy Quinto

<a id='research_question'></a>
# Research Question

Can we find a correlation between cancer rates and socioeconomic status?  More specifically, how does income, education, and employment status affect cancer rates across the United States?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [18]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Reading in the dataset
data_cancer = pd.read_csv('data/cancer_reg.csv')
data_cancer.head()

,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
0,0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657


# Data Cleaning

In [39]:
# Adding County, State columns from info in the 'geography' column
data_cancer[['County', 'State']] = data_cancer['geography'].str.split(',', n=1, expand=True).astype('string')
data_cancer['State'] = data_cancer['State'].str.replace('^.', '', regex=True)
data_cancer.head()

,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,County,State,Region
0,0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",...,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831,Kitsap County,Washington,North West
1,1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",...,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096,Kittitas County,Washington,North West
2,2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",...,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488,Klickitat County,Washington,North West
3,3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",...,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841,Lewis County,Washington,North West
4,4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",...,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657,Lincoln County,Washington,North West


In [52]:
# Assigning a Region to each County based on its state
West = ['California', 'Nevada', 'Hawaii']
NorthWest = ['Alaska', 'Washington', 'Oregon', 'Idaho', 'Montana', 'Wyoming']
SouthWest = ['Utah', 'Colorado', 'Arizona', 'New Mexico', 'Texas', 'Oklahoma']
MidWest = ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri', 'Wisconsin', 'Illinois', 'Indiana', 'Michigan', 'Indiana', 'Kentucky', 'Ohio']
SouthEast = ['Arkansas', 'Louisiana', 'Mississippi', 'Tennessee', 'North Carolina', 'South Carolina', 'Georgia', 'Alabama', 'Florida']
MidAtlantic = ['District of Columbia', 'Virginia', 'West Virginia', 'Delaware', 'Maryland', 'Pennsylvania', 'New Jersey', 'Connecticut', 'New York']
NorthEast = ['Connecticut', 'Rhode Island', 'Massachusetts', 'Vermont', 'New Hampshire', 'Maine']
# Length of all lists combines to 50
print(len(West) + len(NorthWest) + len(SouthWest) + len(MidWest) + len(SouthEast) + len(MidAtlantic) + len(NorthEast))

# Create the 'Region' column
data_cancer['Region'] = data_cancer['State'].apply(lambda x: 'West' if x in West else
                                                         'North West' if x in NorthWest else
                                                         'South West' if x in SouthWest else
                                                         'Mid West' if x in MidWest else
                                                         'South East' if x in SouthEast else
                                                         'Mid Atlantic' if x in MidAtlantic else
                                                         'North East' if x in NorthEast else '')

pd.set_option('display.max_rows', None)
data_cancer.sort_values('State')

53


,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,County,State,Region
1063,1063,406.000000,158,193.8,482.400000,54298,81468,14.4,36.824275,"(51046.4, 54545.6]",...,14.4,75.516016,21.218875,0.516326,0.678529,54.326202,5.800147,Elmore County,Alabama,South East
2684,2684,291.000000,120,175.4,429.900000,45567,57673,17.5,0.000000,"(45201, 48021.6]",...,19.0,95.097903,1.531797,0.143823,0.961705,59.439854,6.894123,Blount County,Alabama,South East
2674,2674,198.000000,78,193.3,487.400000,41574,33115,16.6,0.000000,"(40362.7, 42724.4]",...,18.9,77.347704,11.457155,0.110165,0.059549,52.382359,4.830100,Lawrence County,Alabama,South East
2663,2663,529.000000,238,194.4,449.500000,41324,92596,18.7,21.599205,"(40362.7, 42724.4]",...,17.1,86.926469,9.962582,0.724630,0.219977,50.137263,5.109790,Lauderdale County,Alabama,South East
2653,2653,100.000000,43,202.2,491.600000,34553,13886,20.6,0.000000,"(34218.1, 37413.8]",...,23.1,87.384136,11.073374,0.021227,0.629732,55.539264,7.489134,Lamar County,Alabama,South East
1084,1084,615.000000,264,201.0,472.900000,39904,103057,19.0,0.000000,"(37413.8, 40362.7]",...,19.9,81.252048,15.443402,0.654357,0.697724,50.289481,4.287972,Etowah County,Alabama,South East
2706,2706,452.000000,191,197.4,465.900000,39999,80862,22.5,0.000000,"(37413.8, 40362.7]",...,23.6,64.664710,31.954763,0.451883,0.751501,45.512984,4.578696,Talladega County,Alabama,South East
2685,2685,482.000000,219,192.0,406.900000,41256,156993,25.2,6.369711,"(40362.7, 42724.4]",...,13.6,69.887801,23.244493,3.276550,1.724047,44.562803,4.405337,Lee County,Alabama,South East
1052,1052,322.000000,138,170.6,399.400000,36241,71130,24.0,0.000000,"(34218.1, 37413.8]",...,20.6,86.417234,1.822198,0.308156,6.945461,55.419182,6.414172,DeKalb County,Alabama,South East
2707,2707,248.000000,111,200.5,454.400000,36779,40844,21.3,24.483400,"(34218.1, 37413.8]",...,23.5,70.293782,27.711224,0.036449,0.826185,49.641610,5.462418,Tallapoosa County,Alabama,South East


In [53]:
data_cancer.groupby('Region').count()

,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,County,State
Region,,,,,,,,,,,,,,,,,,,,,
Mid Atlantic,366,366,366,366,366,366,366,366,366,366,...,366,366,366,366,366,366,366,366,366,366
Mid West,1149,1149,1149,1149,1149,1149,1149,1149,1149,1149,...,1149,1149,1149,1149,1149,1149,1149,1149,1149,1149
North East,59,59,59,59,59,59,59,59,59,59,...,59,59,59,59,59,59,59,59,59,59
North West,206,206,206,206,206,206,206,206,206,206,...,206,206,206,206,206,206,206,206,206,206
South East,745,745,745,745,745,745,745,745,745,745,...,745,745,745,745,745,745,745,745,745,745
South West,444,444,444,444,444,444,444,444,444,444,...,444,444,444,444,444,444,444,444,444,444
West,78,78,78,78,78,78,78,78,78,78,...,78,78,78,78,78,78,78,78,78,78


Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION